In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import csv

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 6
hidden_size = 50
num_classes = 15
num_epochs = 10
batch_size = 100
learning_rate = 0.001

TRAIN_DIR = 'D:/PycharmProjects/marchMadness/train'

In [ ]:
def read_train_data(file):
    with open(TRAIN_DIR + '/' + file) as fd:
        reader=csv.reader(fd)
        data=[row for idx, row in enumerate(reader) if idx in (0,1,2)]
        for i in data:
            for x in range(0,len(i)):
                i[x] = float(i[x])         
    return data

def labelTM(data):
    label = data[2][0]
    return label

def labelOPP(data):
    label = data[2][1]
    return label

def label_file(file):
    word_label = file.split('.')[-6]
    if word_label == 'win': 
        x = 1
    elif word_label == 'loss': 
        x = 0
    return x

In [ ]:
def create_train_data():
    training_data = []
    training_labels = []
    training_labels_tm = []
    training_labels_opp = []
    for file in tqdm(os.listdir(TRAIN_DIR)):
        outcome = label_file(file)
        data = read_train_data(file)
        scoreTM = int(labelTM(data))
        scoreTM = round(scoreTM,-1)
        scoreOPP = int(labelOPP(data))
        scoreOPP = round(scoreOPP,-1)
        training_data.append(torch.FloatTensor(data[:2]).unsqueeze(0))
        training_labels_tm.append(scoreTM)
        training_labels_opp.append(scoreOPP)
        training_labels.append(outcome)
    return training_data, training_labels_tm, training_labels_opp, training_labels

training_data, training_labels_tm, training_labels_opp, training_labels = create_train_data()

In [ ]:
training_data = training_data[:-10000]
training_labels = training_labels[:-10000]
training_labels_tm = torch.LongTensor(training_labels_tm[:-10000])
training_labels_opp = torch.LongTensor(training_labels_opp[:-10000])
testing_data = training_data[-10000:]
testing_labels_tm = torch.LongTensor(training_labels_tm[-10000:])
testing_labels_opp = torch.LongTensor(training_labels_opp[-10000:])
testing_labels = torch.LongTensor(training_labels[-10000:])

print('No. of training data: ' + str(len(training_data)))
print('No. of training labels tm: ' + str(len(training_labels_tm)))
print('No. of training labels opp: ' + str(len(training_labels_opp)))
print('No. of training labels outcome: ' + str(len(training_labels)))
print('No. of testing data: ' + str(len(testing_data)))
print('No. of testing labels tm: ' + str(len(testing_labels_tm)))
print('No. of testing labels opp: ' + str(len(testing_labels_opp)))
print('No. of testing labels outcome: ' + str(len(testing_labels)))


In [ ]:
class MyDataset(Dataset):
    def __init__(self, datas, labels1, labels2, labels3):
        self.datas = datas
        self.labels1 = labels1
        self.labels2 = labels2
        self.labels3 = labels3

    def __getitem__(self, index):
        data, target1, target2, target3 = self.datas[index], self.labels1[index], self.labels2[index], self.labels3[index]
        return data, target1, target2

    def __len__(self):
        return len(self.datas)


train_dataloader = DataLoader(MyDataset(training_data, training_labels_tm, training_labels_opp, training_labels), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(MyDataset(testing_data, testing_labels_tm, testing_labels_opp, testing_labels), batch_size=batch_size, shuffle=False)


In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.fc1(x)
        x3 = self.fc1(x)
        x1 = self.relu(x1)
        x2 = self.relu(x2)
        x3 = self.relu(x3)
        x1 = self.fc2(x1)
        x2 = self.fc2(x2)
        x3 = self.fc2(x3)
        return x1,x2,x3

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
# Train the model
total_step = len(train_dataloader)
print(total_step)
for epoch in range(num_epochs):
    for i, (images, target1, target2, target3) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 2*3).to(device)
        target1 = target1.to(device)
        target2 = target2.to(device)
        target3 = target3.to(device)
        
        # Forward pass
        out1, out2, out3 = model(images)
        #print(out1)
        #print('--------------------')
        loss1 = criterion(out1, target1)
        loss2 = criterion(out2, target2)
        loss3 = criterion(out3, target3)
        loss = loss1 + loss2 + loss3
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(epoch+1, num_epochs, i+1, total_step, loss.item()), end='\r')

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, target1, target2 in test_dataloader:
        images = images.reshape(-1, 2*3).to(device)
        target1 = target1.to(device)
        target2 = target2.to(device)
        out1, out2 = model(images)
        _, predicted1 = torch.max(out1.data, 1)
        _, predicted2 = torch.max(out2.data, 1)
        print('-----------------------------------------------------------------------------------------------------')
        print('score tm: ' + str(target1))
        print('score opp: ' + str(target2))
        print('predicted tm: ' + str(predicted1))
        print('predicted op: ' + str(predicted2))
        print('-----------------------------------------------------------------------------------------------------')
        total += target1.size(0) + target2.size(0)
        correct += (predicted1 == target1).sum().item() + (predicted2 == target2).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))